In [ ]:
import os
import numpy as np
import tensorflow as tf
import pickle
import scipy.io as sio

In [ ]:
# from load_data import loading_data
# from net_structure_img import img_net_strucuture
# from net_structure_txt import txt_net_strucuture
# from utils.calc_hammingranking import calc_map

In [ ]:
from datetime import datetime, timedelta
import time
import pdb

In [ ]:
# environmental setting: setting the following parameters based on your experimental environment.
select_gpu = '0'
per_process_gpu_memory_fraction = 0.9

In [ ]:
# data parameters
#DATA_DIR = 'data/FLICKR-25K.mat'
# TRAINING_SIZE = 10000
# QUERY_SIZE = 2000
# DATABASE_SIZE = 18015
# DATA_DIR = '/home/iacvlab/Titir/Work_3/Sketchy_extended_features/' # ---- local
DATA_DIR = '/home/titir/work_sbir/ft_vgg_feat/'   # ------------ cluster
train_=80
val_=10
te_=10

In [ ]:
# hyper-parameters
MAX_ITER = 1
gamma = 1
eta = 1
bit = 64

In [ ]:
filename = 'log/result_' + datetime.now().strftime("%d-%h-%m-%s") + '_' + str(bit) + 'bits_Sketchy_extended.pkl'

In [ ]:
def train_img_net(image_input, cur_f_batch, var, ph, train_x, train_L, lr, train_step_x, mean_pixel_, Sim):
    F = var['F']
    batch_size = var['batch_size']
    num_train = train_x.shape[0]
    # index = range(0, num_train - 1, 1)
    for iter in xrange(num_train / batch_size):
        index = np.random.permutation(num_train)
        ind = index[0: batch_size]
        # ind = index[iter * batch_size: (iter + 1) * batch_size]
        unupdated_ind = np.setdiff1d(range(num_train), ind)
        sample_L = train_L[ind, :]
        image = train_x[ind, :, :, :].astype(np.float64)
        image = image - mean_pixel_.astype(np.float64)

        S = calc_neighbor(sample_L, train_L)
        cur_f = cur_f_batch.eval(feed_dict={image_input: image})
        F[:, ind] = cur_f

        train_step_x.run(feed_dict={ph['S_x']: S, ph['G']: var['G'], ph['b_batch']: var['B'][:, ind],
                                                                ph['F_']: F[:, unupdated_ind], ph['lr']: lr, image_input: image})

    return F





def train_txt_net(text_input, cur_g_batch, var, ph, train_y, train_L, lr, train_step_y, Sim):
    G = var['G']
    batch_size = var['batch_size']
    num_train = train_x.shape[0]
    for iter in xrange(num_train / batch_size):
        index = np.random.permutation(num_train)
        ind = index[0: batch_size]
        unupdated_ind = np.setdiff1d(range(num_train), ind)
        sample_L = train_L[ind, :]
        text = train_y[ind, :].astype(np.float32)
        text = text.reshape([text.shape[0],1,text.shape[1],1])
        S = calc_neighbor(train_L, sample_L)
        cur_g = cur_g_batch.eval(feed_dict={text_input: text})
        G[:, ind] = cur_g

        train_step_y.run(feed_dict={ph['S_y']: S, ph['F']: var['F'], ph['b_batch']: var['B'][:, ind],
                                                                ph['G_']: G[:, unupdated_ind], ph['lr']: lr, text_input: text})
    return G










def calc_neighbor(label_1, label_2):
    Sim = (np.dot(label_1, label_2.transpose()) > 0).astype(int)
    return Sim




def calc_loss(B, F, G, Sim, gamma, eta):
    theta = np.matmul(np.transpose(F), G) / 2
    term1 = np.sum(np.log(1+np.exp(theta)) - Sim * theta)
    term2 = np.sum(np.power((B-F), 2) + np.power(B-G,2))
    term3 = np.sum(np.power(np.matmul(F, np.ones((F.shape[1],1))),2)) + np.sum(np.power(np.matmul(G, np.ones((F.shape[1],1))),2))
    loss = term1 + gamma * term2 + eta * term3
    return loss





def generate_image_code(image_input, cur_f_batch, X, bit, mean_pixel):
    batch_size = 128
    num_data = X.shape[0]
    index = np.linspace(0, num_data - 1, num_data).astype(int)
    B = np.zeros([num_data, bit], dtype=np.float32)
    for iter in xrange(num_data / batch_size + 1):
        ind = index[iter * batch_size : min((iter + 1)*batch_size, num_data)]
        mean_pixel_ = np.repeat(mean_pixel[0][2][:, :, :, np.newaxis], len(ind), axis=3)
        image = X[ind, :, :, :].astype(np.float32) - mean_pixel_.astype(np.float32).transpose(3, 0, 1, 2)
        cur_f = cur_f_batch.eval(feed_dict={image_input: image})
        B[ind, :] = cur_f.transpose()
    B = np.sign(B)
        #pdb.set_trace()
    return B




def generate_text_code(text_input, cur_g_batch, Y, bit):
    batch_size = 128
    num_data = Y.shape[0]
    index = np.linspace(0, num_data - 1, num_data).astype(int)
    B = np.zeros([num_data, bit], dtype=np.float32)
    for iter in xrange(num_data / batch_size + 1):
                #pdb.set_trace()
        ind = index[iter * batch_size : min((iter + 1)*batch_size, num_data)]
        text = Y[ind, :].astype(np.float32)
        text = text.reshape([text.shape[0],1,text.shape[1],1])
        cur_g = cur_g_batch.eval(feed_dict={text_input: text})
        B[ind, :] = cur_g.transpose()
    B = np.sign(B)
        #pdb.set_trace()
    return B






def test_validation(B, query_L, train_L, qBX, qBY):
    mapi2t = calc_map(qBX, B, query_L, train_L)
    mapt2i = calc_map(qBY, B, query_L, train_L)
    return mapi2t, mapt2i





LAYER1_NODE = 8192

def txt_net_strucuture(text_input, dimy, bit):

    W_fc8 = tf.random_normal([1, dimy, 1, LAYER1_NODE], stddev=1.0) * 0.01
    b_fc8 = tf.random_normal([1, LAYER1_NODE], stddev=1.0) * 0.01
    fc1W = tf.Variable(W_fc8)
    fc1b = tf.Variable(b_fc8)

    # ### debugging .......................
    # wb = scipy.io.loadmat('data/wb-text.mat')
    # fc1W = tf.Variable(wb['w1'] * 0.01)
    # fc1b = tf.Variable(wb['b1'] * 0.01)

    conv1 = tf.nn.conv2d(text_input, fc1W, strides=[1, 1, 1, 1], padding='VALID')

    layer1 = tf.nn.relu(tf.nn.bias_add(conv1, tf.squeeze(fc1b)))

    W_fc2 = tf.random_normal([1, 1, LAYER1_NODE, bit], stddev=1.0) * 0.01
    b_fc2 = tf.random_normal([1, bit], stddev=1.0) * 0.01
    fc2W = tf.Variable(W_fc2)
    fc2b = tf.Variable(b_fc2)

    # ### debugging .......................
    # fc2W = tf.Variable(wb['w2'] * 0.01)
    # fc2b = tf.Variable(wb['b2'] * 0.01)
    conv2 = tf.nn.conv2d(layer1, fc2W, strides=[1, 1, 1, 1], padding='VALID')
    output_g = tf.transpose(tf.squeeze(tf.nn.bias_add(conv2, tf.squeeze(fc2b))))
    return output_g






LAYER1_NODE = 8192

def img_net_strucuture(img_input, dimx, bit):

    W_fc8 = tf.random_normal([1, dimx, 1, LAYER1_NODE], stddev=1.0) * 0.01
    b_fc8 = tf.random_normal([1, LAYER1_NODE], stddev=1.0) * 0.01
    fc1W = tf.Variable(W_fc8)
    fc1b = tf.Variable(b_fc8)

    # ### debugging .......................
    # wb = scipy.io.loadmat('data/wb-text.mat')
    # fc1W = tf.Variable(wb['w1'] * 0.01)
    # fc1b = tf.Variable(wb['b1'] * 0.01)

    conv1 = tf.nn.conv2d(img_input, fc1W, strides=[1, 1, 1, 1], padding='VALID')

    layer1 = tf.nn.relu(tf.nn.bias_add(conv1, tf.squeeze(fc1b)))

    W_fc2 = tf.random_normal([1, 1, LAYER1_NODE, bit], stddev=1.0) * 0.01
    b_fc2 = tf.random_normal([1, bit], stddev=1.0) * 0.01
    fc2W = tf.Variable(W_fc2)
    fc2b = tf.Variable(b_fc2)

    # ### debugging .......................
    # fc2W = tf.Variable(wb['w2'] * 0.01)
    # fc2b = tf.Variable(wb['b2'] * 0.01)
    conv2 = tf.nn.conv2d(layer1, fc2W, strides=[1, 1, 1, 1], padding='VALID')
    output_g = tf.transpose(tf.squeeze(tf.nn.bias_add(conv2, tf.squeeze(fc2b))))
    return output_g



In [ ]:
def loading_data(path):
    img_file = sio.loadmat(os.path.join(path,'feat_Sketchy_extended_img.mat'))
    sk_file = sio.loadmat(os.path.join(path,'feat_Sketchy_extended_sk.mat'))
    
    images = img_file['img_feat']
    sketches = sk_file['sk_feat']
    
    img_labels = img_file['img_label'].reshape(images.shape[0],)
    sk_labels = sk_file['sk_label'].reshape(sketches.shape[0],)
    
    return images, img_labels, sketches, sk_labels



def arrange_split_idx(img_label):
    
    unique_img_label=np.unique(img_label)
    num_unique_img_label=unique_img_label.shape[0]
    
    train_idx=np.empty((0,))
    val_idx=np.empty((0,))
    te_idx=np.empty((0,))

    for i in range(num_unique_img_label):
        select_label=unique_img_label[i]
        select_label_idx=np.where(select_label==img_label)
        num_select_label_samples=select_label_idx[0].shape[0]

        num_train=np.ceil(train_*num_select_label_samples/100).astype(int)
        select_train_label_idx=select_label_idx[0][:num_train]

        no_train_label_idx=np.setdiff1d(select_label_idx,select_train_label_idx)
        num_no_train_sample=no_train_label_idx.shape[0]

        num_val=np.ceil(val_*num_select_label_samples/100).astype(int)
        select_val_label_idx=no_train_label_idx[:num_val]

        select_te_label_idx=np.setdiff1d(no_train_label_idx,select_val_label_idx)
        
        train_idx=np.append(train_idx,select_train_label_idx,axis=0).astype(int)
        val_idx=np.append(val_idx,select_val_label_idx,axis=0).astype(int)
        te_idx=np.append(te_idx,select_te_label_idx,axis=0).astype(int)
        
        
    return train_idx, val_idx, te_idx



def get_one_hot(targets, nb_classes):
    targets=targets.astype(int)
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])




def arrange_data(X_train,l_X_train,Y_train,l_Y_train):
    
    unique_l_X=np.unique(l_X_train)
    unique_l_Y=np.unique(l_Y_train)
    
    x_dim=X_train.shape[1]
    y_dim=Y_train.shape[1]
    
    X_selected_train=np.empty((0,x_dim))
    Y_selected_train=np.empty((0,y_dim))
    L_selected_train=np.empty((0,))
    
    for i in range(unique_l_X.shape[0]):
        l_X=unique_l_X[i]
        
        X_idx=np.where(l_X==l_X_train)
        Y_idx=np.where(l_X==l_Y_train)
        
        num_X_idx=X_idx[0].shape[0]
        num_Y_idx=Y_idx[0].shape[0]
        
        num_sample=np.minimum(num_X_idx,num_Y_idx)
        
        X_selected_train=np.append(X_selected_train,X_train[X_idx[0][0:num_sample],:],axis=0)
        Y_selected_train=np.append(Y_selected_train,Y_train[Y_idx[0][0:num_sample],:],axis=0)
        L_selected_train=np.append(L_selected_train,l_X_train[X_idx[0][0:num_sample]],axis=0)
        
    
    return X_selected_train,Y_selected_train,L_selected_train
        
    
    



def split_data(images, tags, labels):
    X = {}
    X['query'] = images[0:QUERY_SIZE,:,:,:]
    X['train'] = images[QUERY_SIZE:TRAINING_SIZE + QUERY_SIZE,:,:,:]
    X['retrieval'] = images[QUERY_SIZE:DATABASE_SIZE + QUERY_SIZE,:,:,:]

    Y = {}
    Y['query'] = tags[0:QUERY_SIZE,:]
    Y['train'] = tags[QUERY_SIZE:TRAINING_SIZE + QUERY_SIZE,:]
    Y['retrieval'] = tags[QUERY_SIZE:DATABASE_SIZE + QUERY_SIZE,:]

    L = {}
    L['query'] = labels[0:QUERY_SIZE,:]
    L['train'] = labels[QUERY_SIZE:TRAINING_SIZE + QUERY_SIZE,:]
    L['retrieval'] = labels[QUERY_SIZE:DATABASE_SIZE + QUERY_SIZE,:]

    return X, Y, L

In [ ]:
images, img_labels, sketches, sk_labels = loading_data(DATA_DIR)

num_classes = np.unique(img_labels).shape[0]

img_train_idx, img_val_idx,img_te_idx=arrange_split_idx(img_labels)

img_one_hot=get_one_hot(img_labels,num_classes)

X_train=images[img_train_idx,:]
l_X_train=img_labels[img_train_idx]


X_val=images[img_val_idx,:]
l_X_val=img_labels[img_val_idx]

X_te=images[img_te_idx,:]
l_X_te=img_labels[img_te_idx]








sk_train_idx, sk_val_idx, sk_te_idx=arrange_split_idx(sk_labels)

num_classes = np.unique(sk_labels).shape[0]

sk_one_hot=get_one_hot(sk_labels,num_classes)


Y_train=sketches[img_train_idx,:]
l_Y_train=sk_labels[img_train_idx]

Y_val=sketches[img_val_idx,:]
l_Y_val=sk_labels[img_val_idx]

Y_te=sketches[img_te_idx,:]
l_Y_te=sk_labels[img_te_idx]





X_selected_train,Y_selected_train,L_selected_train=arrange_data(X_train,l_X_train,Y_train,l_Y_train)





X = {}
X['query'] = X_te
X['train'] = X_selected_train
X['retrieval'] = X_te

Y = {}
Y['query'] = Y_te
Y['train'] = Y_selected_train
Y['retrieval'] = Y_te

L = {}
L['query'] = get_one_hot(l_X_te,num_classes)
L['train'] = get_one_hot(L_selected_train,num_classes)
L['retrieval'] = get_one_hot(l_Y_te,num_classes)



print('...loading and splitting data finish')


In [ ]:
# if __name__ == '__main__':
#     images, tags, labels = loading_data(DATA_DIR)
#     ydim = tags.shape[1]

#     X, Y, L = split_data(images, tags, labels)
#     print('...loading and splitting data finish')
    
gpuconfig = tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=per_process_gpu_memory_fraction))
os.environ["CUDA_VISIBLE_DEVICES"] = select_gpu


xdim=X_selected_train.shape[1]
ydim=Y_selected_train.shape[1]

batch_size = 8

with tf.Graph().as_default(), tf.Session(config=gpuconfig) as sess:
    
#     # construct image network
#     image_input = tf.placeholder(tf.float32, (None,) + (224, 224, 3))
#     net, _meanpix = img_net_strucuture(image_input, bit)
#             # pdb.set_trace()
#     mean_pixel_ = np.repeat(_meanpix[0][2][:, :, :, np.newaxis], batch_size, axis=3).transpose(3,0,1,2)
#     cur_f_batch = tf.transpose(net['fc8'])
    
    
    # construct image network
    image_input = tf.placeholder(tf.float32, (None,) + (1, xdim, 1))
    cur_f_batch = img_net_strucuture(image_input, xdim, bit)
    
    
    # construct text network
    text_input = tf.placeholder(tf.float32, (None,) + (1, ydim, 1))
    cur_g_batch = txt_net_strucuture(text_input, ydim, bit)

    # training DCMH algorithm
    train_L = L['train']
    train_x = X['train']
    train_y = Y['train']

    query_L = L['query']
    query_x = X['query']
    query_y = Y['query']

    retrieval_L = L['retrieval']
    retrieval_x = X['retrieval']
    retrieval_y = Y['retrieval']
    num_train = train_x.shape[0]

    Sim = calc_neighbor(train_L, train_L)

    var = {}
    # lr = np.logspace(-1.5, -3, MAX_ITER)
    lr = np.linspace(np.power(10, -1.5), np.power(10, -6.), MAX_ITER)

    var['lr'] = lr
    var['batch_size'] = batch_size

    var['F'] = np.random.randn(bit, num_train)
    var['G'] = np.random.randn(bit, num_train)
    var['B'] = np.sign(var['F']+var['G'])

    unupdated_size = num_train - batch_size
    var['unupdated_size'] = unupdated_size

    ph = {}
    ph['lr'] = tf.placeholder('float32', (), name='lr')
    ph['S_x'] = tf.placeholder('float32', [batch_size, num_train], name='pS_x')
    ph['S_y'] = tf.placeholder('float32', [num_train, batch_size], name='pS_y')
    ph['F'] = tf.placeholder('float32', [bit, num_train], name='pF')
    ph['G'] = tf.placeholder('float32', [bit, num_train], name='pG')
    ph['F_'] = tf.placeholder('float32', [bit, unupdated_size], name='unupdated_F')
    ph['G_'] = tf.placeholder('float32', [bit, unupdated_size], name='unupdated_G')
    ph['b_batch'] = tf.placeholder('float32', [bit, batch_size], name='b_batch')
    ph['ones_'] = tf.constant(np.ones([unupdated_size, 1], 'float32'))
    ph['ones_batch'] = tf.constant(np.ones([batch_size, 1], 'float32'))

    theta_x = 1.0 / 2 * tf.matmul(tf.transpose(cur_f_batch), ph['G'])
    theta_y = 1.0 / 2 * tf.matmul(tf.transpose(ph['F']), cur_g_batch)

    logloss_x = -tf.reduce_sum(tf.multiply(ph['S_x'], theta_x) - tf.log(1.0 + tf.exp(theta_x)))
    quantization_x = tf.reduce_sum(tf.pow((ph['b_batch'] - cur_f_batch), 2))
    balance_x = tf.reduce_sum(tf.pow(tf.matmul(cur_f_batch, ph['ones_batch']) + tf.matmul(ph['F_'], ph['ones_']), 2))
    loss_x = tf.div(logloss_x + gamma * quantization_x + eta * balance_x, float(num_train * batch_size))

    logloss_y = -tf.reduce_sum(tf.multiply(ph['S_y'], theta_y) - tf.log(1.0 + tf.exp(theta_y)))
    quantization_y = tf.reduce_sum(tf.pow((ph['b_batch'] - cur_g_batch), 2))
    balance_y = tf.reduce_sum(tf.pow(tf.matmul(cur_g_batch, ph['ones_batch']) + tf.matmul(ph['G_'], ph['ones_']), 2))
    loss_y = tf.div(logloss_y + gamma * quantization_y + eta * balance_y, float(num_train * batch_size))

    optimizer = tf.train.GradientDescentOptimizer(ph['lr'])

    gradient_x = optimizer.compute_gradients(loss_x)
    gradient_y = optimizer.compute_gradients(loss_y)
    train_step_x = optimizer.apply_gradients(gradient_x)
    train_step_y = optimizer.apply_gradients(gradient_y)
    sess.run(tf.global_variables_initializer())
    loss_ = calc_loss(var['B'], var['F'], var['G'], Sim, gamma, eta)
    print('...epoch: %3d, loss: %3.3f' % (0, loss_))
    result = {}
    result['loss'] = []
    result['imapi2t'] = []
    result['imapt2i'] = []

    print('...training procedure starts')

    for epoch in range(MAX_ITER):
        lr = var['lr'][epoch]
        # update F
        var['F'] = train_img_net(image_input, cur_f_batch, var, ph,  train_x, train_L, lr, train_step_x, mean_pixel_, Sim)

        # update G
        var['G'] = train_txt_net(text_input, cur_g_batch, var, ph, train_y, train_L, lr, train_step_y, Sim)

        # update B
        var['B'] = np.sign(gamma * (var['F'] + var['G']))

        # calculate loss
        loss_ = calc_loss(var['B'], var['F'], var['G'], Sim, gamma, eta)
        print('...epoch: %3d, loss: %3.3f, comment: update B' % (epoch + 1, loss_))

        result['loss'].append(loss_)
    print('...training procedure finish')
    qBX = generate_image_code(image_input, cur_f_batch, query_x, bit, _meanpix)
            #print(qBX)
    qBY = generate_text_code(text_input, cur_g_batch, query_y, bit)
            #print(qBY)
    rBX = generate_image_code(image_input, cur_f_batch, retrieval_x, bit, _meanpix)
            #print(rBX)
    rBY = generate_text_code(text_input, cur_g_batch, retrieval_y, bit)
            #print(rBY)

    print("generated image and text codes")

    mapi2t = calc_map(qBX, rBY, query_L, retrieval_L)
    mapt2i = calc_map(qBY, rBX, query_L, retrieval_L)

    print('...test map: map(i->t): %3.3f, map(t->i): %3.3f' % (mapi2t, mapt2i))

    result['mapi2t'] = mapi2t
    result['mapt2i'] = mapt2i
    result['lr'] = lr

    fp = open(filename, 'wb')
    pickle.dump(result, fp)

    fp.close()

